## Objective 1: Extracting helsinki city-center image and level the coordinates of crossroads 

In [3]:
##let's read the image data from wms api
from owslib.wms import WebMapService
import os

In [4]:
wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.3.0')

In [3]:
##These coordinates locate focssed image nearby Central Railway Station
minX=24.941622
minY=60.170030
maxX=24.947408
maxY=60.173158
min_x_cord=[]
max_x_cord=[]
min_y_cord=[]
max_y_cord=[]
img_name=[]
a=np.linspace(minX,maxX,10)
b=np.linspace(minY,maxY,10)
counter=0
imgpath="C:/Users/35841/images/citycenterfocussed500by500/"
image_files=[f for f in os.listdir(imgpath)]


In [4]:
##Creating patches for the image
for i in range(0,len(a)-1):
    for j in range(0,len(b)-1):
        image_name='center_'+str(counter)
        if not image_files:
            img=wms.getmap(layers=["Ortoilmakuva_2019_5cm"],srs="EPSG:4123",bbox=(a[i],b[j],a[i+1],b[j+1]),size=(500,500),format='image/png',transparent=True)
            out = open("images/citycenterfocussed500by500/"+image_name+'.png', 'wb')
            out.write(img.read())
            out.close()
        min_x_cord.append(a[i])
        min_y_cord.append(b[j])
        max_x_cord.append(a[i+1])
        max_y_cord.append(b[j+1])
        img_name.append(image_name)
        counter+=1
                   

In [5]:
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas as pd

ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'


In [6]:
##After labelling the images using labelImg "https://github.com/tzutalin/labelImg", the code below crates the annotaion file
annotations = []
classes = set([])
DATASET_DIR="C:/Users/35841/images/xml_citycenter_focussed/"
for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  file_name = None

  for elem in root:
    if elem.tag == 'filename':
      file_name = os.path.join(DATASET_DIR, elem.text)

    if elem.tag == 'object':
      obj_name = None
      coords = []
      for subelem in elem:
        if subelem.tag == 'name':
          obj_name = subelem.text
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            coords.append(subsubelem.text)
      item = [file_name] + coords + [obj_name]
      annotations.append(item)
      classes.add(obj_name)

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))


In [7]:
##Let's create the dataframe that consists the image name and the corresponding latitude and longitude columns
df = pd.DataFrame(list(zip(img_name, min_x_cord,max_x_cord,min_y_cord,max_y_cord)), columns =['img_name', 'min_x_cord','max_x_cord','min_y_cord','max_y_cord'], dtype = float)
print(df.shape)
df.head(5)

(81, 5)


,img_name,min_x_cord,max_x_cord,min_y_cord,max_y_cord
0,center_0,24.941622,24.942265,60.170030,60.170378
1,center_1,24.941622,24.942265,60.170378,60.170725
2,center_2,24.941622,24.942265,60.170725,60.171073
3,center_3,24.941622,24.942265,60.171073,60.171420
4,center_4,24.941622,24.942265,60.171420,60.171768


In [8]:
##Creating another dataframe from the annotation.csv obtained after labelling the images
crossroad_df=pd.read_csv("annotations.csv",names=["img_name","x_min","y_min","x_max","y_max","label"])
crossroad_df.head(5)

,img_name,x_min,y_min,x_max,y_max,label
0,C:/Users/35841/images/xml_citycenter_focussed/...,1,1,496,497,Unknown
1,C:/Users/35841/images/xml_citycenter_focussed/...,2,1,497,500,Unknown
2,C:/Users/35841/images/xml_citycenter_focussed/...,2,2,500,498,Unknown
3,C:/Users/35841/images/xml_citycenter_focussed/...,5,3,493,499,Unknown
4,C:/Users/35841/images/xml_citycenter_focussed/...,30,364,256,499,crossroad


In [9]:
crossroad_df["img_name"]=crossroad_df["img_name"].str.replace("C:/Users/35841/images/xml_citycenter_focussed/","").str.replace(".png","")
crossroad_df["count"]=crossroad_df["img_name"].str.extract((r'(\d+)'))
crossroad_df["count"]=crossroad_df["count"].astype(int)
crossroad_df=crossroad_df.sort_values(by=["count"])
crossroad_df.head(5)

,img_name,x_min,y_min,x_max,y_max,label,count
0,center_0,1,1,496,497,Unknown,0
1,center_1,2,1,497,500,Unknown,1
16,center_2,344,390,491,479,crossroad,2
15,center_2,333,291,484,365,crossroad,2
29,center_3,1,1,499,500,Unknown,3


In [36]:
numeric_label=crossroad_df["label"].str.replace("Unknown","0").str.replace("crossroad","1").astype(int)
test_df=crossroad_df.copy()
test_df["numeric_label"]=numeric_label
test_df.head(5)

,img_name,x_min,y_min,x_max,y_max,label,count,numeric_label
0,center_0,1,1,496,497,Unknown,0,0
1,center_1,2,1,497,500,Unknown,1,0
16,center_2,344,390,491,479,crossroad,2,1
15,center_2,333,291,484,365,crossroad,2,1
29,center_3,1,1,499,500,Unknown,3,0


In [49]:
test_grouped=test_df[["img_name","numeric_label"]].groupby(["img_name"]).sum()
test_grouped

,numeric_label
img_name,
center_0,0
center_1,0
center_10,0
center_11,0
center_12,1
...,...
center_78,0
center_79,0
center_8,0


In [10]:

new_df=pd.merge(crossroad_df,df,how='left', on=["img_name"])
new_df.head(5)

,img_name,x_min,y_min,x_max,y_max,label,count,min_x_cord,max_x_cord,min_y_cord,max_y_cord
0,center_0,1,1,496,497,Unknown,0,24.941622,24.942265,60.170030,60.170378
1,center_1,2,1,497,500,Unknown,1,24.941622,24.942265,60.170378,60.170725
2,center_2,344,390,491,479,crossroad,2,24.941622,24.942265,60.170725,60.171073
3,center_2,333,291,484,365,crossroad,2,24.941622,24.942265,60.170725,60.171073
4,center_3,1,1,499,500,Unknown,3,24.941622,24.942265,60.171073,60.171420


In [11]:
##calculating width of a patch in terms of latitude and logitude
x_cordinate_interval=max_x_cord[1]-min_x_cord[1]
y_cordinate_interval=max_y_cord[1]-min_y_cord[1]
print(x_cordinate_interval)
print(y_cordinate_interval)
print("***********************************************")

##calculating the ratio of coordinate to image size
image_size=(500,500)
one_ybounding_translation=y_cordinate_interval/image_size[0]
one_xbounding_translation=x_cordinate_interval/image_size[0]
print(one_ybounding_translation)
print(one_xbounding_translation)


0.0006428888888905249
0.00034755555555676665
***********************************************
6.951111111135333e-07
1.2857777777810497e-06


In [12]:
#translation to coordinates and finally adding coordinates of each label to the dataframe
new_df["minLat"]=new_df["min_x_cord"]+new_df["x_min"]*one_xbounding_translation
new_df["minLon"]=new_df["max_y_cord"]-new_df["y_max"]*one_ybounding_translation
new_df["maxLat"]=new_df["min_x_cord"]+new_df["x_max"]*one_xbounding_translation
new_df["maxLon"]=new_df["max_y_cord"]-new_df["y_min"]*one_ybounding_translation

In [13]:
new_df.head(5)

,img_name,x_min,y_min,x_max,y_max,label,count,min_x_cord,max_x_cord,min_y_cord,max_y_cord,minLat,minLon,maxLat,maxLon
0,center_0,1,1,496,497,Unknown,0,24.941622,24.942265,60.170030,60.170378,24.941623,60.170032,24.942260,60.170377
1,center_1,2,1,497,500,Unknown,1,24.941622,24.942265,60.170378,60.170725,24.941625,60.170378,24.942261,60.170724
2,center_2,344,390,491,479,crossroad,2,24.941622,24.942265,60.170725,60.171073,24.942064,60.170740,24.942253,60.170802
3,center_2,333,291,484,365,crossroad,2,24.941622,24.942265,60.170725,60.171073,24.942050,60.170819,24.942244,60.170870
4,center_3,1,1,499,500,Unknown,3,24.941622,24.942265,60.171073,60.171420,24.941623,60.171073,24.942264,60.171420


In [14]:
##verify coordinate nmapping for cross_road in images center_2.jpg, center_76.pg
img=wms.getmap(layers=["Ortoilmakuva_2019_5cm"],srs="EPSG:4123",bbox=(24.942064,60.170740,24.942253,60.170802),size=(500,500),format='image/png',transparent=True)
out = open('images/check1.png', 'wb')
out.write(img.read())
out.close()

img=wms.getmap(layers=["Ortoilmakuva_2019_5cm"],srs="EPSG:4123",bbox=(24.947315,60.171682,24.947404,60.171767),size=(500,500),format='image/png',transparent=True)
out = open('images/check2.png', 'wb')
out.write(img.read())
out.close()

In [ ]:
##We may be interested to know the specific coordinate rather than bounding bx for each level which can be obatianed as
#new_df["mean_lat"]=new_df["minLat"]*0.5+new_df["maxLat"]*0.5
#new_df["mean_lon"]=new_df["minLon"]*0.5+new_df["maxLon"]*0.5

## Objective 2: ML Implementations

In [15]:
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [16]:
dataset=np.ndarray(shape=(len(image_files),500,500,3),dtype=np.float32)


In [56]:
i = 0
for _file in test_grouped.index:
    #print(_file)
    img = load_img(imgpath+ _file+".png")  # this is a PIL image
    img.thumbnail(image_size)
    # Convert to Numpy Array
    x = img_to_array(img)  
    #x = x.reshape((3, 120, 160))
    # Normalize
    x = (x - 128.0) / 128.0
    dataset[i] = x
    i += 1
   

In [146]:
dataset.shape

(81, 500, 500, 3)

In [130]:
from sklearn.model_selection import train_test_split
#Splitting 
X_train, X_test, y_train, y_test = train_test_split(dataset,test_grouped["numeric_label"].values, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [131]:
datagen = ImageDataGenerator(
        rotation_range=90,
        vertical_flip=True,
        horizontal_flip=True)
datagen.fit(X_train)
val_datagen = ImageDataGenerator(
        rotation_range=90,
       vertical_flip=True,
        horizontal_flip=True)
val_datagen.fit(X_val)



In [132]:
datagen

In [133]:
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping

In [134]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=X_train[0].shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('linear'))
model.add(Dropout(0.5))
model.add(Dense(1))


C:\Users\35841\Anaconda3\envs\mllearn\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(500, 500,...)`
  
C:\Users\35841\Anaconda3\envs\mllearn\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
C:\Users\35841\Anaconda3\envs\mllearn\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [141]:
sgd = SGD(lr=0.01, decay=1e-7, momentum=.9)
    
early_stop = EarlyStopping(monitor='val_loss', verbose=1, mode='auto')

model.compile(loss='mse', optimizer='rmsprop')

model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), 
                    samples_per_epoch=len(X_train), validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
                    nb_val_samples=len(X_val), 
                    nb_epoch=5, 
                    verbose=1
                    )


C:\Users\35841\Anaconda3\envs\mllearn\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\35841\Anaconda3\envs\mllearn\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., verbose=1, steps_per_epoch=2, epochs=5, validation_steps=9)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
2/2 [==============================] - 14s 7s/step - loss: 81.8746 - val_loss: 292.8813
Epoch 2/5
2/2 [==============================] - 14s 7s/step - loss: 197.7567 - val_loss: 0.9121
Epoch 3/5
2/2 [==============================] - 14s 7s/step - loss: 1.5192 - val_loss: 0.8397
Epoch 4/5
2/2 [==============================] - 14s 7s/step - loss: 1.4426 - val_loss: 0.7788
Epoch 5/5
2/2 [==============================] - 13s 7s/step - loss: 1.5342 - val_loss: 0.7577


In [142]:
predictions = model.predict(X_test)
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 37ms/step


2.1761019229888916

In [143]:
print("{0:5} {1:10} {2:8}".format("True", "Predicted", "Error"))
for i in range(len(predictions[:10])):
    print("{0:5} {1:10} {2:8}".format(y_test[i], int(predictions[i]), abs(y_test[i] - int(predictions[i]))))

True  Predicted  Error   
    0          0        0
    4          0        4
    0          0        0
    0          0        0
    0          0        0
    2          0        2
    0          0        0
    2          0        2


In [82]:
X_train.shape

(64, 500, 500, 3)

In [83]:
X_test.shape

(17, 500, 500, 3)

In [21]:
## more data
minX=24.916459
minY=60.229677
maxX=24.966731 
maxY=60.231718
min_x_cord=[]
max_x_cord=[]
min_y_cord=[]
max_y_cord=[]
img_name=[]
a=np.linspace(minX,maxX,7)
b=np.linspace(minY,maxY,7)
counter=0
imgpath="C:/Users/35841/images/moredata/"
image_files=[f for f in os.listdir(imgpath)]



In [22]:
##Creating patches for the image
for i in range(0,len(a)-1):
    for j in range(0,len(b)-1):
        image_name='outside_'+str(counter)
        if not image_files:
            img=wms.getmap(layers=["Ortoilmakuva_2019_5cm"],srs="EPSG:4123",bbox=(a[i],b[j],a[i+1],b[j+1]),size=(50,50),format='image/png',transparent=True)
            out = open("C:/Users/35841/images/moredata/"+image_name+'.png', 'wb')
            out.write(img.read())
            out.close()
        min_x_cord.append(a[i])
        min_y_cord.append(b[j])
        max_x_cord.append(a[i+1])
        max_y_cord.append(b[j+1])
        img_name.append(image_name)
        counter+=1